In [1]:
from Bio.PDB import MMCIFParser,PDBIO
from pathlib import Path

In [2]:
parser = MMCIFParser()
io = PDBIO()

def cif_to_pdb(directory):
    for file in directory.glob("*.cif"):
        model_number = file.stem.split("_")[-1]
        cif = file.stem.split("_")[-3]
        structure = parser.get_structure(model_number, file)
        io.set_structure(structure)
        output_file = cif+"_model"+model_number+".pdb"
        io.save(output_file)

In [ ]:
for directory in Path("../").iterdir():
    if directory.is_dir() and ("cid" in directory.name or "cif" in directory.name):
        cif_to_pdb(directory)
        print(f"Converted files in {directory}")

Converted files in ../fold_2025_06_12_13_13_mf444991_cidb
Converted files in ../fold_2025_06_12_13_12_mf444974_cida


In [ ]:
from Bio.PDB import PDBParser
from Bio.PDB.DSSP import DSSP
import os

p = PDBParser()

alpha_codes = {'H', 'G', 'I'}  # alpha-helix, 3-10 helix, pi-helix
beta_codes = {'E', 'B'}        # beta-sheet, beta-bridge

for dir in os.listdir("."):
    if not os.path.isdir(dir):
        continue

    total_alpha = 0
    total_beta = 0
    total_other = 0
    total_residues = 0
    protein_count = 0
    num = dir.split("-")[0]
    protein = dir.split("-")[1]
    pdb_path = f"{num}-{protein}/{protein.lower()}_model0.pdb"
    if not os.path.exists(pdb_path):
        continue
    structure = p.get_structure(protein, pdb_path)
    model = structure[0]
    dssp = DSSP(model, pdb_path, dssp="mkdssp")
    alpha = beta = other = 0
    for key in dssp.keys():
        ss = dssp[key][2]
        if ss in alpha_codes:
            alpha += 1
        elif ss in beta_codes:
            beta += 1
        else:
            other += 1
    n = alpha + beta + other
    if n == 0:
        continue
    total_alpha += alpha
    total_beta += beta
    total_other += other
    total_residues += n
    protein_count += 1

    if total_residues > 0:
        print(protein)
        print(f"alpha(%): {100 * total_alpha / total_residues:.2f}")
        print(f"beta(%): {100 * total_beta / total_residues:.2f}")
        print(f"other(%): {100 * total_other / total_residues:.2f}")


cifA
alpha(%): 66.12
beta(%): 1.22
other(%): 32.65
cidB
alpha(%): 37.18
beta(%): 21.56
other(%): 41.26
cifB
alpha(%): 34.63
beta(%): 26.29
other(%): 39.08
cidA
alpha(%): 58.57
beta(%): 2.45
other(%): 38.98
